In [1]:
def read_file(k):
  file=open('data1/short_1_dummy.fasta','r')
  reads=[]
  lines=file.readlines()
  for i in range(0,len(lines)):
    if(i%2!=0):
      lines[i]=lines[i].rstrip(lines[i][-1])
      reads.append(lines[i])
    else:
      pass
  file.close()
  return reads

In [2]:
def read_to_kmers(reads,k):
  kmers_list={}
  for read in reads:
    for i in range(len(read)-k+1):
      kmer=read[i:i+k]
      if not kmer in kmers_list.keys():
        kmers_list[kmer]=1
      else:
        kmers_list[kmer]+=1
  return kmers_list

In [3]:
def get_debruijn_edges_from_kmers(kmers):
    # store edges as tuples in a set
    edges = set()
    edge_label=[]
    
    # compare each (k-1)mer
    for k1 in kmers:
        for k2 in kmers:
            if k1 != k2:            
                # if they overlap then add to edges
                if k1[1:] == k2[:-1]:
                    edges.add((k1[:-1], k2[:-1]))
                if k1[:-1] == k2[1:]:
                    edges.add((k2[:-1], k1[:-1]))

    return edges

In [5]:
import toyplot
def plot_debruijn_graph(edges, width=700, height=700):
    "returns a toyplot graph from an input of edges"
    graph = toyplot.graph(
        [i[0] for i in edges],
        [i[1] for i in edges],
        width=width,
        height=height,
        tmarker=">", 
        vsize=25,
        vstyle={"stroke": "white", "stroke-width": 8, "fill": "white"},
        vlstyle={"font-size": "11px"},
        estyle={"stroke": "white", "stroke-width": 2},
        layout=toyplot.layout.FruchtermanReingold(edges=toyplot.layout.CurvedEdges()))
    return graph

In [6]:
k = input("Enter the value of k : ")
k = int(k)
reads=read_file(k)

In [7]:
kmers = read_to_kmers(reads,k)
#print(kmers)

In [8]:
edges = get_debruijn_edges_from_kmers(kmers)
#print(edges)

In [9]:
plot_debruijn_graph(edges)

(<toyplot.canvas.Canvas at 0x221de777070>,
 <toyplot.mark.Graph at 0x221edd1eee0>)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="700.0px" height="700.0px" viewBox="0 0 700.0 700.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t64361e55b0374c1cb8001d8dfcf2e913"> AA AC AG AT CA CC CG CT GA GC GG GT TA TC TG TT

In [19]:
def get_dbg_graph_from_edges(edges):
    fin_dict={}
    for edge in edges:
        if edge[0] not in fin_dict:
            fin_dict[edge[0]] = [edge[1]]
        else:
            fin_dict[edge[0]].append(edge[1])
    
    return (fin_dict)


graph = get_dbg_graph_from_edges(edges)

In [26]:
visited = []  # List for visited nodes.
queue = []  # Initialize a queue
#print(graph)

BFS_traversal = []

def bfs(visited, graph, node, k):  # function for BFS
    visited.append(node)
    queue.append(node)
    contig = ""

    while queue:          # Creating loop to visit each node
        m = queue.pop(0)
        #print(m, end=" ")
        BFS_traversal.append(m)

        for neighbour in graph[m]:
            if len(graph[m]) > 1 and contig != "":
                print(contig, end=", ")
                contig = ""

            else:
                if(len(contig) < k):
                    contig += neighbour
                else:
                    contig += neighbour[-1]
            if neighbour not in visited:
                visited.append(neighbour)
                queue.append(neighbour)

print("The contigs are: ")
bfs(visited, graph, list(graph.keys())[0], k)
print("\n\nBFS traversal of the de Bruijn Graph is: ")
print(" -> ".join(BFS_traversal))

The contigs are: 
TC, TA, CT, CA, GA, GG, AA, AC, TG, TT, TA, TC, CT, CA, AA, AC, GG, GA, AT, AG, GG, GA, CT, CA, AA, AC, TT, TG, CT, CA, GT, GC, 

BFS traversal of the de Bruijn Graph is: 
GT -> TC -> TG -> TA -> TT -> CT -> CC -> CA -> CG -> GA -> GG -> GC -> AA -> AT -> AC -> AG
